<a href="https://colab.research.google.com/github/tztechno/cc_archive/blob/main/dino_lightglue_colmap_gs_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DINO/LIGHTGLUE/COLMAP/GS 01

In [1]:

#サイズの異なる画像を扱う
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import sys
import subprocess

def run_cmd(cmd, check=True, capture=False):
    """Run command with better error handling"""
    print(f"Running: {' '.join(cmd)}")
    result = subprocess.run(
        cmd,
        capture_output=capture,
        text=True,
        check=False
    )
    if check and result.returncode != 0:
        print(f"❌ Command failed with code {result.returncode}")
        if capture:
            print(f"STDOUT: {result.stdout}")
            print(f"STDERR: {result.stderr}")
    return result

def setup_environment():
    """
    Colab environment setup for Gaussian Splatting + LightGlue + pycolmap
    Python 3.12 compatible version (v8)
    """

    print("🚀 Setting up COLAB environment (v8 - Python 3.12 compatible)")

    WORK_DIR = "/content/gaussian-splatting"

    # =====================================================================
    # STEP 0: NumPy FIX (Python 3.12 compatible)
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 0: Fix NumPy (Python 3.12 compatible)")
    print("="*70)

    # Python 3.12 requires numpy >= 1.26
    run_cmd([sys.executable, "-m", "pip", "uninstall", "-y", "numpy"])
    run_cmd([sys.executable, "-m", "pip", "install", "numpy==1.26.4"])

    # sanity check
    run_cmd([sys.executable, "-c", "import numpy; print('NumPy:', numpy.__version__)"])

    # =====================================================================
    # STEP 1: System packages (Colab)
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 1: System packages")
    print("="*70)

    run_cmd(["apt-get", "update", "-qq"])
    run_cmd([
        "apt-get", "install", "-y", "-qq",
        "colmap",
        "build-essential",
        "cmake",
        "git",
        "libopenblas-dev",
        "xvfb"
    ])

    # virtual display (COLMAP / OpenCV safety)
    os.environ["QT_QPA_PLATFORM"] = "offscreen"
    os.environ["DISPLAY"] = ":99"
    subprocess.Popen(
        ["Xvfb", ":99", "-screen", "0", "1024x768x24"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )

    # =====================================================================
    # STEP 2: Clone Gaussian Splatting
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 2: Clone Gaussian Splatting")
    print("="*70)

    if not os.path.exists(WORK_DIR):
        run_cmd([
            "git", "clone", "--recursive",
            "https://github.com/graphdeco-inria/gaussian-splatting.git",
            WORK_DIR
        ])
    else:
        print("✓ Repository already exists")

    # =====================================================================
    # STEP 3: Python packages (FIXED ORDER & VERSIONS)
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 3: Python packages (VERBOSE MODE)")
    print("="*70)

    # ---- PyTorch (Colab CUDA対応) ----
    print("\n📦 Installing PyTorch...")
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "torch", "torchvision", "torchaudio"
    ])

    # ---- Core utils ----
    print("\n📦 Installing core utilities...")
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "opencv-python",
        "pillow",
        "imageio",
        "imageio-ffmpeg",
        "plyfile",
        "tqdm",
        "tensorboard"
    ])

    # ---- transformers (NumPy 1.26 compatible) ----
    print("\n📦 Installing transformers (NumPy 1.26 compatible)...")
    # Install transformers with proper dependencies
    run_cmd([
        sys.executable, "-m", "pip", "install",
        "transformers==4.40.0"
    ])

    # ---- LightGlue stack (GITHUB INSTALL) ----
    print("\n📦 Installing LightGlue stack...")

    # Install kornia first
    run_cmd([sys.executable, "-m", "pip", "install", "kornia"])

    # Install h5py (sometimes needed)
    run_cmd([sys.executable, "-m", "pip", "install", "h5py"])

    # Install matplotlib (LightGlue dependency)
    run_cmd([sys.executable, "-m", "pip", "install", "matplotlib"])

    # Install LightGlue directly from GitHub (more reliable)
    print("  Installing LightGlue from GitHub...")
    run_cmd([sys.executable, "-m", "pip", "install",
            "git+https://github.com/cvg/LightGlue.git"])

    # Install pycolmap
    run_cmd([sys.executable, "-m", "pip", "install", "pycolmap"])

    # =====================================================================
    # STEP 4: Build GS submodules
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 4: Build Gaussian Splatting submodules")
    print("="*70)

    submodules = {
        "diff-gaussian-rasterization":
            "https://github.com/graphdeco-inria/diff-gaussian-rasterization.git",
        "simple-knn":
            "https://github.com/camenduru/simple-knn.git"
    }

    for name, repo in submodules.items():
        print(f"\n📦 Installing {name}...")
        path = os.path.join(WORK_DIR, "submodules", name)
        if not os.path.exists(path):
            run_cmd(["git", "clone", repo, path])
        run_cmd([sys.executable, "-m", "pip", "install", path])

    # =====================================================================
    # STEP 5: Detailed Verification
    # =====================================================================
    print("\n" + "="*70)
    print("STEP 5: Detailed Verification")
    print("="*70)

    # NumPy (verify version first)
    print("\n🔍 Testing NumPy...")
    try:
        import numpy as np
        print(f"  ✓ NumPy: {np.__version__}")
    except Exception as e:
        print(f"  ❌ NumPy failed: {e}")

    # PyTorch
    print("\n🔍 Testing PyTorch...")
    try:
        import torch
        print(f"  ✓ PyTorch: {torch.__version__}")
        print(f"  ✓ CUDA available: {torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"  ✓ CUDA version: {torch.version.cuda}")
    except Exception as e:
        print(f"  ❌ PyTorch failed: {e}")

    # transformers
    print("\n🔍 Testing transformers...")
    try:
        import transformers
        print(f"  ✓ transformers version: {transformers.__version__}")
        from transformers import AutoModel
        print(f"  ✓ AutoModel import: OK")
    except Exception as e:
        print(f"  ❌ transformers failed: {e}")
        print(f"  Attempting detailed diagnosis...")
        result = run_cmd([
            sys.executable, "-c",
            "import transformers; print(transformers.__version__)"
        ], capture=True)
        print(f"  Output: {result.stdout}")
        print(f"  Error: {result.stderr}")

    # LightGlue
    print("\n🔍 Testing LightGlue...")
    try:
        from lightglue import LightGlue, ALIKED
        print(f"  ✓ LightGlue: OK")
        print(f"  ✓ ALIKED: OK")
    except Exception as e:
        print(f"  ❌ LightGlue failed: {e}")
        print(f"  Attempting detailed diagnosis...")
        result = run_cmd([
            sys.executable, "-c",
            "from lightglue import LightGlue"
        ], capture=True)
        print(f"  Output: {result.stdout}")
        print(f"  Error: {result.stderr}")

    # pycolmap
    print("\n🔍 Testing pycolmap...")
    try:
        import pycolmap
        print(f"  ✓ pycolmap: OK")
    except Exception as e:
        print(f"  ❌ pycolmap failed: {e}")

    # kornia
    print("\n🔍 Testing kornia...")
    try:
        import kornia
        print(f"  ✓ kornia: {kornia.__version__}")
    except Exception as e:
        print(f"  ❌ kornia failed: {e}")

    print("\n" + "="*70)
    print("✅ SETUP COMPLETE")
    print("="*70)
    print(f"Working dir: {WORK_DIR}")

    return WORK_DIR


if __name__ == "__main__":
    setup_environment()

🚀 Setting up COLAB environment (v8 - Python 3.12 compatible)

STEP 0: Fix NumPy (Python 3.12 compatible)
Running: /usr/bin/python3 -m pip uninstall -y numpy
Running: /usr/bin/python3 -m pip install numpy==1.26.4
Running: /usr/bin/python3 -c import numpy; print('NumPy:', numpy.__version__)

STEP 1: System packages
Running: apt-get update -qq
Running: apt-get install -y -qq colmap build-essential cmake git libopenblas-dev xvfb

STEP 2: Clone Gaussian Splatting
Running: git clone --recursive https://github.com/graphdeco-inria/gaussian-splatting.git /content/gaussian-splatting

STEP 3: Python packages (VERBOSE MODE)

📦 Installing PyTorch...
Running: /usr/bin/python3 -m pip install torch torchvision torchaudio

📦 Installing core utilities...
Running: /usr/bin/python3 -m pip install opencv-python pillow imageio imageio-ffmpeg plyfile tqdm tensorboard

📦 Installing transformers (NumPy 1.26 compatible)...
Running: /usr/bin/python3 -m pip install transformers==4.40.0

📦 Installing LightGlue sta

In [3]:
!ls

drive  gaussian-splatting  sample_data


In [4]:
import os

# gaussian-splattingディレクトリに移動
%cd /content/gaussian-splatting

# ファイルが既に存在するかチェックしてダウンロード（重複防止）
files = ['database.py', 'h5_to_db.py', 'metric.py']
base_url = 'https://huggingface.co/stpete2/imc25_utils/resolve/main/'

for file in files:
    if not os.path.exists(file):
        !wget -q {base_url + file}
        print(f"✓ {file} ダウンロード完了")
    else:
        print(f"✓ {file} 既に存在")



/content/gaussian-splatting
✓ database.py ダウンロード完了
✓ h5_to_db.py ダウンロード完了
✓ metric.py ダウンロード完了


In [5]:
# これでimportできる
from database import COLMAPDatabase, image_ids_to_pair_id
from h5_to_db import add_keypoints, add_matches
from metric import *

In [6]:
"""
Gaussian Splatting Pipeline
Simple and robust pipeline: LightGlue → COLMAP → Gaussian Splatting
"""

import os
import sys
import gc
import h5py
import numpy as np
import torch
import torch.nn.functional as F
from tqdm import tqdm
from pathlib import Path
import subprocess

# LightGlue
from lightglue import ALIKED, LightGlue
from lightglue.utils import load_image

# Transformers for DINO
from transformers import AutoImageProcessor, AutoModel



# ============================================================================
# Configuration
# ============================================================================
class Config:
    # Feature extraction
    N_KEYPOINTS = 8192
    IMAGE_SIZE = 1536

    # Pair selection
    GLOBAL_TOPK = 200
    MIN_MATCHES = 10
    RATIO_THR = 1.2

    # Paths
    DINO_MODEL = "facebook/dinov2-base"  # Change if using local path

    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# ============================================================================
# Step 1: Image Pair Selection (DINO + ALIKED local verify)
# ============================================================================

def load_torch_image(fname, device):
    """Load image as torch tensor"""
    from PIL import Image
    import torchvision.transforms as T

    img = Image.open(fname).convert('RGB')
    transform = T.Compose([
        T.ToTensor(),
    ])
    return transform(img).unsqueeze(0).to(device)

def extract_dino_global(image_paths, model_path, device):
    """Extract DINO global descriptors"""
    print("\n=== Extracting DINO Global Features ===")

    processor = AutoImageProcessor.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path).eval().to(device)

    global_descs = []
    for img_path in tqdm(image_paths):
        img = load_torch_image(img_path, device)
        with torch.no_grad():
            inputs = processor(images=img, return_tensors="pt", do_rescale=False).to(device)
            outputs = model(**inputs)
            desc = F.normalize(outputs.last_hidden_state[:, 1:].max(dim=1)[0], dim=1, p=2)
            global_descs.append(desc.cpu())

    global_descs = torch.cat(global_descs, dim=0)

    del model
    torch.cuda.empty_cache()
    gc.collect()

    return global_descs

def build_topk_pairs(global_feats, k, device):
    """Build top-k similar pairs from global features"""
    g = global_feats.to(device)
    sim = g @ g.T
    sim.fill_diagonal_(-1)

    N = sim.size(0)
    k = min(k, N - 1)

    topk_indices = torch.topk(sim, k, dim=1).indices.cpu()

    pairs = []
    for i in range(N):
        for j in topk_indices[i]:
            j = j.item()
            if i < j:
                pairs.append((i, j))

    return list(set(pairs))

def extract_aliked_features(image_paths, device):
    """Extract ALIKED local features"""
    print("\n=== Extracting ALIKED Local Features ===")

    extractor = ALIKED(
        model_name="aliked-n16",
        max_num_keypoints=Config.N_KEYPOINTS,
        detection_threshold=0.01,
        resize=Config.IMAGE_SIZE
    ).eval().to(device)

    features = []
    for img_path in tqdm(image_paths):
        img = load_torch_image(img_path, device)
        with torch.no_grad():
            feats = extractor.extract(img)
            kpts = feats['keypoints'].reshape(-1, 2).cpu()
            descs = feats['descriptors'].reshape(len(kpts), -1).cpu()
            features.append({'keypoints': kpts, 'descriptors': descs})

    del extractor
    torch.cuda.empty_cache()
    gc.collect()

    return features

def verify_pairs_locally(pairs, features, device, threshold=Config.MIN_MATCHES):
    """Verify pairs using local descriptor matching"""
    print("\n=== Verifying Pairs with Local Features ===")

    verified = []
    for i, j in tqdm(pairs):
        desc1 = features[i]['descriptors'].to(device)
        desc2 = features[j]['descriptors'].to(device)

        if len(desc1) == 0 or len(desc2) == 0:
            continue

        # Simple mutual nearest neighbor
        dist = torch.cdist(desc1, desc2, p=2)
        min_dist = dist.min(dim=1)[0]
        n_matches = (min_dist < Config.RATIO_THR).sum().item()

        if n_matches >= threshold:
            verified.append((i, j))

    return verified

def get_image_pairs(image_paths):
    """Main pair selection pipeline"""
    device = Config.DEVICE

    # 1. DINO global
    global_feats = extract_dino_global(image_paths, Config.DINO_MODEL, device)
    pairs = build_topk_pairs(global_feats, Config.GLOBAL_TOPK, device)

    print(f"Initial pairs from global features: {len(pairs)}")

    # 2. ALIKED local
    features = extract_aliked_features(image_paths, device)

    # 3. Local verification
    verified_pairs = verify_pairs_locally(pairs, features, device)

    print(f"Verified pairs: {len(verified_pairs)}")

    return verified_pairs, features

In [8]:
# ============================================================================
# Step 2: Feature Matching (ALIKED + LightGlue)
# ============================================================================

def match_pairs_lightglue(image_paths, pairs, features, output_dir):
    """
    Match image pairs using LightGlue
    """
    print("\n=== Matching with LightGlue ===")

    os.makedirs(output_dir, exist_ok=True)
    keypoints_path = os.path.join(output_dir, 'keypoints.h5')
    matches_path = os.path.join(output_dir, 'matches.h5')

    # 既存のh5ファイルを削除
    if os.path.exists(keypoints_path):
        os.remove(keypoints_path)
    if os.path.exists(matches_path):
        os.remove(matches_path)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    extractor = ALIKED(max_num_keypoints=4096, detection_threshold=0.2, nms_radius=2).eval().to(device)
    matcher = LightGlue(features='aliked').eval().to(device)

    # featuresの形式に応じて処理
    if isinstance(features, dict):
        all_keypoints = features['keypoints']
        all_descriptors = features['descriptors']
    elif isinstance(features, list):
        all_keypoints = [f['keypoints'] for f in features]
        all_descriptors = [f['descriptors'] for f in features]
    else:
        raise ValueError(f"Unsupported features type: {type(features)}")

    # Save keypoints - 画像ファイル名（拡張子なし）をキーにする
    with h5py.File(keypoints_path, 'w') as f_kp:
        for idx, img_path in enumerate(tqdm(image_paths, desc="Saving keypoints")):
            img_name = os.path.splitext(os.path.basename(img_path))[0]

            kp = all_keypoints[idx]
            if torch.is_tensor(kp):
                kp = kp.cpu().numpy()
            f_kp.create_dataset(img_name, data=kp)

    # Match pairs
    with h5py.File(matches_path, 'w') as f_match:
        for idx1, idx2 in tqdm(pairs, desc="Matching"):
            with torch.no_grad():
                kp0 = all_keypoints[idx1]
                kp1 = all_keypoints[idx2]
                desc0 = all_descriptors[idx1]
                desc1 = all_descriptors[idx2]

                if isinstance(kp0, np.ndarray):
                    kp0 = torch.from_numpy(kp0).float().to(device)
                    kp1 = torch.from_numpy(kp1).float().to(device)
                    desc0 = torch.from_numpy(desc0).float().to(device)
                    desc1 = torch.from_numpy(desc1).float().to(device)
                else:
                    kp0 = kp0.float().to(device)
                    kp1 = kp1.float().to(device)
                    desc0 = desc0.float().to(device)
                    desc1 = desc1.float().to(device)

                feats0 = {
                    'keypoints': kp0.unsqueeze(0) if kp0.dim() == 2 else kp0,
                    'descriptors': desc0.unsqueeze(0) if desc0.dim() == 2 else desc0,
                }
                feats1 = {
                    'keypoints': kp1.unsqueeze(0) if kp1.dim() == 2 else kp1,
                    'descriptors': desc1.unsqueeze(0) if desc1.dim() == 2 else desc1,
                }

                matches01 = matcher({'image0': feats0, 'image1': feats1})

                if 'matches0' in matches01:
                    matches0 = matches01['matches0'].cpu().numpy()
                    if matches0.ndim > 1:
                        matches0 = matches0[0]
                    valid = matches0 > -1
                    matches = np.stack([np.where(valid)[0], matches0[valid]], axis=1)
                elif 'matches' in matches01:
                    m = matches01['matches']
                    if isinstance(m, list):
                        matches = np.array(m)
                    elif hasattr(m, 'cpu'):
                        matches = m.cpu().numpy()
                    else:
                        matches = np.array(m)
                else:
                    continue

                if len(matches) > 0:
                    img_name1 = os.path.splitext(os.path.basename(image_paths[idx1]))[0]
                    img_name2 = os.path.splitext(os.path.basename(image_paths[idx2]))[0]
                    pair_key = f"{img_name1}_{img_name2}"
                    f_match.create_dataset(pair_key, data=matches)

    print(f"✓ Matches saved to {matches_path}")


In [9]:
# ============================================================================
# Step 3: Import to COLMAP
# ============================================================================

def import_to_colmap(image_dir, feature_dir, database_path, single_camera=True):
    """
    Import features and matches to COLMAP database

    Args:
        image_dir: 画像ディレクトリ
        feature_dir: 特徴量ディレクトリ
        database_path: データベースパス
        single_camera: 全画像が同じサイズならTrue（統一済みならTrue）
    """
    print("\n=== Creating COLMAP Database ===")

    # 既存のデータベースを削除
    if os.path.exists(database_path):
        os.remove(database_path)
        print(f"✓ Removed existing database")

    db = COLMAPDatabase.connect(database_path)
    db.create_tables()

    print(f"Single camera mode: {single_camera}")

    # 拡張子を自動検出
    image_files = [f for f in os.listdir(image_dir)
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    if not image_files:
        raise ValueError(f"No images found in {image_dir}")

    first_image = sorted(image_files)[0]
    img_ext = os.path.splitext(first_image)[1]
    print(f"Detected image extension: '{img_ext}'")

    # ★★★ ここを修正: SIMPLE_RADIAL → PINHOLE ★★★
    # キーポイントを追加
    fname_to_id = add_keypoints(
        db,
        feature_dir,
        image_dir,
        img_ext,  # 拡張子を渡す
        'PINHOLE',  # ← カメラモデルを PINHOLE に変更！
        single_camera=single_camera
    )

    # マッチを追加
    add_matches(db, feature_dir, fname_to_id)

    db.commit()
    db.close()

    print(f"✓ Database created: {database_path}")

# ============================================================================
# Step 4: Run COLMAP Mapper
# ============================================================================

def run_colmap_mapper(database_path, image_dir, output_dir):
    """
    Run COLMAP mapper with verbose output
    """
    print("\n=== Running COLMAP Reconstruction ===")
    os.makedirs(output_dir, exist_ok=True)
    cmd = [
        'colmap', 'mapper',
        '--database_path', database_path,
        '--image_path', image_dir,
        '--output_path', output_dir,
        '--Mapper.ba_refine_focal_length', '0',      # ← '1' を '0' に変更
        '--Mapper.ba_refine_principal_point', '0',   # ← '1' を '0' に変更
        '--Mapper.ba_refine_extra_params', '0',      # ← '1' を '0' に変更
        '--Mapper.min_num_matches', '15',
        '--Mapper.init_min_num_inliers', '50',
        '--Mapper.max_num_models', '1',
        '--Mapper.num_threads', '16',
    ]
    print(f"Command: {' '.join(cmd)}\n")
    # リアルタイム出力
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    for line in process.stdout:
        print(line, end='')
    process.wait()
    if process.returncode == 0:
        model_dir = os.path.join(output_dir, '0')
        if os.path.exists(model_dir):
            print(f"\n✓ COLMAP reconstruction complete: {model_dir}")
            return model_dir
    raise RuntimeError("COLMAP reconstruction failed")

In [10]:
# ============================================================================
# Step 5: Convert to Gaussian Splatting Format (if needed)
# ============================================================================

def convert_to_gs_format(colmap_model_dir, output_dir):
    """
    Verify COLMAP output and prepare path for Gaussian Splatting

    Args:
        colmap_model_dir: COLMAPのsparse/0ディレクトリ
                         例: /content/output/colmap/sparse/0
        output_dir: 出力ベースディレクトリ

    Returns:
        colmap_parent_dir: Gaussian Splattingに渡すパス
                          例: /content/output/colmap (sparse/を含む親)
    """
    print("\n=== Verifying COLMAP Model for Gaussian Splatting ===")

    import pycolmap
    reconstruction = pycolmap.Reconstruction(colmap_model_dir)

    print(f"Registered images: {len(reconstruction.images)}")
    print(f"3D points: {len(reconstruction.points3D)}")

    # Gaussian Splattingに必要なファイルを確認
    required_files = ['cameras.bin', 'images.bin', 'points3D.bin']
    for file in required_files:
        file_path = os.path.join(colmap_model_dir, file)
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Required file not found: {file}")
        print(f"  ✓ {file}")

    # sparse/0の親ディレクトリを返す
    # /content/output/colmap/sparse/0 → /content/output/colmap
    colmap_parent_dir = os.path.dirname(os.path.dirname(colmap_model_dir))

    print(f"\n✓ COLMAP model ready for Gaussian Splatting")
    print(f"  Source path: {colmap_parent_dir}")

    return colmap_parent_dir

In [11]:
def train_gaussian_splatting(colmap_dir, image_dir, output_dir, iterations=30000):
    """
    Train Gaussian Splatting model

    Args:
        colmap_dir: COLMAPの親ディレクトリ（sparse/を含む）
                   例: /content/output/colmap
        image_dir: トレーニング画像ディレクトリ
                  例: /content/output/processed_images
        output_dir: GS出力ディレクトリ
        iterations: トレーニング回数

    Returns:
        gs_output_dir: Gaussian Splattingの出力先
    """
    print("\n=== Training Gaussian Splatting ===")

    gs_output_dir = os.path.join(output_dir, 'gs_output')
    os.makedirs(gs_output_dir, exist_ok=True)

    # Gaussian Splattingのディレクトリ構造を確認
    sparse_dir = os.path.join(colmap_dir, 'sparse', '0')
    if not os.path.exists(sparse_dir):
        raise FileNotFoundError(f"COLMAP sparse directory not found: {sparse_dir}")

    print(f"COLMAP sparse model: {sparse_dir}")
    print(f"Training images: {image_dir}")
    print(f"Output: {gs_output_dir}")
    print(f"Iterations: {iterations}")

    # Gaussian Splattingのコマンド
    cmd = [
        'python', 'train.py',
        '-s', colmap_dir,           # sparse/を含む親ディレクトリ
        '--images', image_dir,       # 画像ディレクトリを明示
        '-m', gs_output_dir,         # モデル出力先
        '--iterations', str(iterations),
        '--test_iterations', '7000', str(iterations),
        '--save_iterations', '7000', str(iterations),
    ]

    print(f"\nCommand: {' '.join(cmd)}\n")

    result = subprocess.run(cmd, capture_output=True, text=True)

    print(result.stdout)
    if result.stderr:
        print("STDERR:", result.stderr)

    if result.returncode != 0:
        raise RuntimeError("Gaussian Splatting training failed")

    # PLYファイルの存在確認
    ply_path = os.path.join(gs_output_dir, 'point_cloud', f'iteration_{iterations}', 'point_cloud.ply')
    if os.path.exists(ply_path):
        size_mb = os.path.getsize(ply_path) / (1024 * 1024)
        print(f"\n✓ Training complete!")
        print(f"  PLY file: {ply_path}")
        print(f"  Size: {size_mb:.2f} MB")
    else:
        print(f"⚠️  Warning: PLY file not found at expected location")

    return gs_output_dir

In [12]:
def preprocess_images_square(input_dir, output_dir, size=1024, background='black'):
    """
    全画像を正方形に統一（アスペクト比保持、パディング追加）

    Args:
        input_dir: 入力画像ディレクトリ
        output_dir: 出力ディレクトリ
        size: 正方形のサイズ (default: 1024)
        background: 'black' or 'white' or 'blur'
    """
    from PIL import Image, ImageFilter
    import os
    from tqdm import tqdm

    print(f"\n=== Preprocessing to {size}x{size} Square Images ===")

    os.makedirs(output_dir, exist_ok=True)

    image_files = sorted([
        f for f in os.listdir(input_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])

    stats = {
        'total': len(image_files),
        'landscape': 0,
        'portrait': 0,
        'square': 0,
        'resized': 0,
    }

    for img_file in tqdm(image_files, desc="Converting to square"):
        img_path = os.path.join(input_dir, img_file)
        img = Image.open(img_path).convert('RGB')

        width, height = img.size

        # 統計
        if width > height:
            stats['landscape'] += 1
        elif width < height:
            stats['portrait'] += 1
        else:
            stats['square'] += 1

        # 長辺を基準にリサイズ
        max_dim = max(width, height)
        if max_dim != size:
            scale = size / max_dim
            new_width = int(width * scale)
            new_height = int(height * scale)
            img = img.resize((new_width, new_height), Image.LANCZOS)
            stats['resized'] += 1
        else:
            new_width, new_height = width, height

        # 背景の作成
        if background == 'black':
            canvas = Image.new('RGB', (size, size), (0, 0, 0))
        elif background == 'white':
            canvas = Image.new('RGB', (size, size), (255, 255, 255))
        elif background == 'blur':
            # ぼかした画像を背景に（高級感）
            canvas = img.resize((size, size), Image.LANCZOS)
            canvas = canvas.filter(ImageFilter.GaussianBlur(radius=20))
        else:
            canvas = Image.new('RGB', (size, size), (0, 0, 0))

        # 中央配置
        offset_x = (size - new_width) // 2
        offset_y = (size - new_height) // 2
        canvas.paste(img, (offset_x, offset_y))

        # 保存
        output_path = os.path.join(output_dir, img_file)
        canvas.save(output_path, quality=95, optimize=True)

    print(f"\n✓ 処理完了:")
    print(f"  総画像数: {stats['total']}")
    print(f"  横長: {stats['landscape']} / 縦長: {stats['portrait']} / 正方形: {stats['square']}")
    print(f"  リサイズ: {stats['resized']}")
    print(f"  出力サイズ: {size}x{size}")

    return output_dir

In [13]:
def main_pipeline(image_dir, output_dir, square_size=1024):
    """
    Complete pipeline: Images → Square → COLMAP → Gaussian Splatting
    """
    print("="*70)
    print("Gaussian Splatting Preparation Pipeline")
    print("="*70)

    # Step 0: 画像を正方形に統一
    processed_dir = os.path.join(output_dir, 'processed_images')
    processed_image_dir = preprocess_images_square(image_dir, processed_dir, size=square_size)

    # Setup paths
    feature_dir = os.path.join(output_dir, 'features')
    colmap_dir = os.path.join(output_dir, 'colmap')
    database_path = os.path.join(colmap_dir, 'database.db')
    sparse_dir = os.path.join(colmap_dir, 'sparse')

    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(colmap_dir, exist_ok=True)

    # Get image paths
    image_paths = sorted([
        os.path.join(processed_image_dir, f)
        for f in os.listdir(processed_image_dir)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ])

    print(f"\n📸 Found {len(image_paths)} images")

    # Step 1: Get image pairs
    pairs, features = get_image_pairs(image_paths)

    # Step 2: Match with LightGlue
    match_pairs_lightglue(image_paths, pairs, features, feature_dir)

    # Step 3: Import to COLMAP (single_camera=True: 全画像同じサイズ)
    import_to_colmap(processed_image_dir, feature_dir, database_path, single_camera=True)

    # Step 4: Run COLMAP Reconstruction
    model_dir = run_colmap_mapper(database_path, processed_image_dir, sparse_dir)

    # Step 5: Verify and prepare for GS
    colmap_parent = convert_to_gs_format(model_dir, output_dir)

    # Step 6: Train Gaussian Splatting
    gs_output = train_gaussian_splatting(
        colmap_dir=colmap_parent,
        image_dir=processed_image_dir,
        output_dir=output_dir,
        iterations=2000
    )

    print("\n" + "="*70)
    print("✅ Complete Pipeline Finished!")
    print("="*70)
    print(f"\nGaussian Splatting model: {gs_output}")

    return gs_output



# 使用例
if __name__ == "__main__":
    IMAGE_DIR = "/content/drive/MyDrive/your_folder/grand_place"  # バラバラの観光写真
    OUTPUT_DIR = "/content/output"

    gs_output = main_pipeline(IMAGE_DIR, OUTPUT_DIR, square_size=1024)



Gaussian Splatting Preparation Pipeline

=== Preprocessing to 1024x1024 Square Images ===


Converting to square: 100%|██████████| 80/80 [01:02<00:00,  1.29it/s]
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



✓ 処理完了:
  総画像数: 80
  横長: 64 / 縦長: 16 / 正方形: 0
  リサイズ: 79
  出力サイズ: 1024x1024

📸 Found 80 images

=== Extracting DINO Global Features ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

...

✓ Matches saved to /content/output/features/matches.h5

=== Creating COLMAP Database ===
Single camera mode: True
Detected image extension: '.jpg'


100%|██████████| 80/80 [00:00<00:00, 2367.82it/s]


カメラ作成: PINHOLE, f=1024.0, cx=512.0, cy=512.0


Adding matches: 100%|██████████| 3151/3151 [00:00<00:00, 3398.78it/s]


ストリーミング出力は最後の 5000 行に切り捨てられました。
Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.304355e+05    0.00e+00    6.65e+04   0.00e+00   0.00e+00  1.00e+04        0    2.87e-01    8.70e-01
   1  3.296866e+05    7.49e+02    1.10e+02   5.63e-01   1.00e+00  3.00e+04        1    7.94e-01    1.66e+00
   2  3.296857e+05    8.85e-01    3.50e+01   9.18e-02   1.03e+00  9.00e+04        1    7.92e-01    2.46e+00
   3  3.296856e+05    1.09e-01    1.83e+01   2.19e-02   1.04e+00  2.70e+05        1    7.82e-01    3.24e+00
   4  3.296856e+05    2.37e-02    7.96e+00   8.70e-03   1.11e+00  8.10e+05        1    8.02e-01    4.04e+00
   5  3.296856e+05    3.80e-03    3.63e+00   3.21e-03   1.20e+00  2.43e+06        1    7.92e-01    4.83e+00
   6  3.296856e+05    4.11e-04    1.52e+00   1.21e-03   1.25e+00  7.29e+06        1    7.84e-01    5.62e+00
   7  3.296856e+05    3.54e-05    4.61e-01   3.65e-04   1.27e+00  2.19e+07    

In [14]:
import pycolmap

sparse_dir = "/content/output/colmap/sparse/0"
reconstruction = pycolmap.Reconstruction(sparse_dir)

print("=== Camera Models in COLMAP ===")
for camera_id, camera in reconstruction.cameras.items():
    # model_nameではなくmodelを使用
    print(f"Camera {camera_id}: {camera.model}")
    print(f"  Width: {camera.width}, Height: {camera.height}")
    print(f"  Params: {camera.params}")
    print()

    # カメラモデルタイプの数値を文字列に変換
    model_names = {
        0: 'SIMPLE_PINHOLE',
        1: 'PINHOLE',
        2: 'SIMPLE_RADIAL',
        3: 'RADIAL',
        4: 'OPENCV',
        5: 'OPENCV_FISHEYE',
        6: 'FULL_OPENCV',
        7: 'FOV',
        8: 'SIMPLE_RADIAL_FISHEYE',
        9: 'RADIAL_FISHEYE',
        10: 'THIN_PRISM_FISHEYE'
    }

    model_type = camera.model.value if hasattr(camera.model, 'value') else camera.model
    model_name = model_names.get(model_type, f"UNKNOWN({model_type})")
    print(f"  Model name: {model_name}")

    if model_name not in ['PINHOLE', 'SIMPLE_PINHOLE']:
        print(f"  ❌ NOT SUPPORTED by Gaussian Splatting!")

=== Camera Models in COLMAP ===
Camera 1: CameraModelId.PINHOLE
  Width: 1024, Height: 1024
  Params: [1024. 1024.  512.  512.]

  Model name: PINHOLE
